In [1]:
import uproot3 as uproot
import numpy as np
import awkward as ak
import matplotlib.pyplot as plt
import os,sys
import math
from sklearn.cluster import Birch#clustering
#this h4 file has implemented extrapolate points, lack of maching up h4 hits and st3 tracklet extrapolation

For analysis at h4, we need to
1. Build a sorter, find out index of event with detID between 41-46
2. get their position
3. get info from st23, do linear extrapolation to h4, muon approximately match, pion scatter. <br>
st23tracklet_py_st3<br>
st23tracklet_pz_st3<br>
st23tracklet_px_st3<br>

Double_t track_x_CAL = track_x_st3 + (track_px_st3 / track_pz_st3) * (1930. - track_z_st3);<br>
Double_t track_y_CAL = track_y_st3 + (track_py_st3 / track_pz_st3) * (1930. - track_z_st3);<br>
st23tracklet_x_st3<br>
st23tracklet_y_st3<br>
st23tracklet_z_st3<br>
41-44 are y<br>
45, 46 are x

For most particles, their hits ID between 41, 46 record are more tham recorded hits on h4. Since hit_pos only provided with detID, i will use detID method to find out evt that has hits on h4

In [2]:
#--------------------------------------------------------------------------------------------------------------------------------------------
def getData(fname="", procName="Events"):
    kn_dict = uproot.open(fname)["Events"].arrays()
    kn_dict_ak1 = {name.decode(): ak.from_awkward0(array) for name, array in kn_dict.items()}
    kn_events = ak.zip({"Electrons":ak.zip({
                                            "ge":      kn_dict_ak1["ge"],
                                            "gvx":      kn_dict_ak1["gvx"],
                                            "gvy":      kn_dict_ak1["gvy"],
                                            "gvz":      kn_dict_ak1["gvz"],
                                            "gpx":      kn_dict_ak1["gpx"],
                                            "gpy":      kn_dict_ak1["gpy"],
                                            "gpz":      kn_dict_ak1["gpz"],
                                        }),
                        "Hits":ak.zip({
                                      "detID":   kn_dict_ak1["hit_detID"],#detector ID for each hit, look for 41-46
                                      "edep":    kn_dict_ak1["hit_edep"],
                                      "elmID":   kn_dict_ak1["hit_elmID"],#EMCal cell ID
                                      "hit_pos":  kn_dict_ak1["hit_pos"],#then find the position of hits(use index) with ID bet41-46
                                        }),
                        "track":ak.zip({
                                      "x":   kn_dict_ak1["track_x_CAL"],
                                      "y":    kn_dict_ak1["track_y_CAL"],
                                      "ID":    kn_dict_ak1["eventID"],
                                      "pz":    kn_dict_ak1["track_pz_st1"],
                                        }),
                        "st23":ak.zip({
                                      "ntrack23":   kn_dict_ak1["n_st23tracklets"],
                                      "px":   kn_dict_ak1["st23tracklet_px_st3"],
                                      "py":   kn_dict_ak1["st23tracklet_py_st3"],
                                      "pz":   kn_dict_ak1["st23tracklet_pz_st3"],
                                      "x":   kn_dict_ak1["st23tracklet_x_st3"],
                                      "y":   kn_dict_ak1["st23tracklet_y_st3"],
                                      "z":   kn_dict_ak1["st23tracklet_z_st3"],
                                        }),
                       }, depth_limit=1)
    return kn_events

In [3]:
def h4_sort(raw_ID, raw_pos):#hard coded for detID 41-46
    indices = []
    detID = []
    h4_pos = []
    for index, value in enumerate(raw_ID):
        if 41 <= value <= 46:
            indices.append(index)
            detID.append(value)
            h4_pos.append(raw_pos[index])
    return indices, detID, h4_pos

In [47]:
def extrpl_h4(x, y, z, px, py, pz, ID):
    IDpos = {41: 2130.27, 42: 2146.45, 43: 2200.44, 44: 2216.62, 45: 2251.71, 46: 2234.29}
    extrapolated_data = []
    for item in ID:
        extrapolated_values = []
        if 41 <= item <= 44:
            for i in range(len(y)):
                extrapolated_values.append(y[i] + (py[i] / pz[i]) * (IDpos[item] - z[i]))
        elif 45 <= item <= 46:
            for i in range(len(x)):
                extrapolated_values.append(x[i] + (px[i] / pz[i]) * (IDpos[item] - z[i]))
        extrapolated_data.append(extrapolated_values)
    return extrapolated_data

In [5]:
os.chdir("/Users/dwong/Desktop/DQ_Dowling/ntuple_1000")
file="muon_520_1000eve.root"
dq_events = getData(file)

IDpos={41: 2130.27, 42: 2146.45, 43:2200.44 , 44:2216.62 , 45:2251.71 , 46: 2234.29}

ID = dq_events["Hits"].detID.tolist()
raw_pos = dq_events["Hits"].hit_pos.tolist()
st3_x= dq_events["st23"].x.tolist()
st3_y= dq_events["st23"].y.tolist()
st3_z= dq_events["st23"].z.tolist()
st3_px= dq_events["st23"].px.tolist()
st3_py= dq_events["st23"].py.tolist()
st3_pz= dq_events["st23"].pz.tolist()

h4_info=[]
extrap=[]
for i in range(len(ID)):#in level of each event
    indices, detID, pos_1D=h4_sort(ID[i], raw_pos[i])
    
    extrap.append(extrpl_h4(st3_x[i], st3_y[i], st3_z[i], st3_px[i], st3_py[i], st3_pz[i], detID))
    h4_info.append([indices, detID, pos_1D])
    

    

AttributeError: module 'awkward' has no attribute 'from_awkward0'

In [70]:
print(extrap[998])
print()
print(st3_x[998])
print()
print(h4_info[998])

[[-0.16493632135284653, 24.114943336435864, 19.479425591924908], [5.250771245547437, -4.751098963985397, -4.383032760247491], [5.268347346096218, -4.916947786479671, -4.616275544553947], [5.3269959041078785, -5.47035804151465, -5.39456837055554], [5.344572004656659, -5.636206864008924, -5.6278111548619965]]

[1.0240103006362915, 14.86052417755127, 11.12347412109375]

[[50, 51, 52, 53, 54], [46, 41, 42, 43, 44], [9.164667129516602, 9.466787338256836, 11.228084564208984, 10.405887603759766, 10.171592712402344]]


In [71]:
extrap

[[[-7.917503923745565],
  [-8.93484503102031],
  [-8.93484503102031],
  [-9.046471272193612],
  [-9.418949687579685],
  [-9.530575928752986]],
 [[-2.0650266464152196],
  [14.28495223432744],
  [14.324543910393121],
  [14.456654824533187],
  [14.49624650059887]],
 [[12.746675539765352], [12.294495354206726], [12.257425128181831]],
 [[-31.71032508179368], [27.572245423051832], [28.03815020711346]],
 [[10.38679801461972], [17.507259653088028], [17.409785369602766]],
 [[-7.246560779829375],
  [-7.246560779829375],
  [-7.11831554109898],
  [-7.11831554109898],
  [-7.11831554109898],
  [2.5357666056746817],
  [2.5357666056746817],
  [2.5114822873095015],
  [2.5114822873095015],
  [2.5114822873095015],
  [2.5114822873095015],
  [2.5114822873095015],
  [2.5114822873095015],
  [2.4304495092788403],
  [2.4304495092788403],
  [2.4304495092788403],
  [2.4304495092788403],
  [2.4304495092788403],
  [2.40616519091366],
  [2.40616519091366],
  [2.40616519091366],
  [2.40616519091366],
  [2.4061651909